In [1]:
# Show all Jupyter cell outputs, NOT just the last cell's output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import pandas as pd
import requests, json 
from bs4 import BeautifulSoup as soup


#####                   Getting posts from website(s) and load into a pandas dataframe
       1. n_pages specifies the number of pages to be queried. one page contains 10 posts
       2. Uncomment only one Web-page you want to collect information from

In [2]:
# web_page = "https://www.mwebantu.news"
# web_page = "https://www.lusakatimes.com"
# web_page = "https://www.zambiawatchdog.com"
# web_page = "https://zambianeye.com"
web_page = "https://www.zambianobserver.com"

n_pages = 10 #only get 10 pages for demonstration

data = pd.DataFrame({})
for i in range(1,n_pages+1):
    url= web_page+"/wp-json/wp/v2/posts?page="+str(i)
#     url= web_page+"/wp-json/wp/v2/categories?page="+str(i)
    r = requests.get(url)
    data = pd.concat([data,pd.DataFrame(r.json())],ignore_index=True, sort=False)
    print(i)

1
2
3
4
5
6
7
8
9
10


###### The code in the next cell will be used to load data from Excell
        for normal execution, kindly comment it

In [3]:
# data = pd.read_excel('mwebantu_posts.xlsx')

In [4]:
data.columns
data.sample(3)

Index(['_links', 'amp_validity', 'author', 'categories', 'comment_status',
       'content', 'date', 'date_gmt', 'excerpt', 'featured_media', 'format',
       'guid', 'id', 'jetpack_featured_media_url',
       'jetpack_publicize_connections', 'jetpack_shortlink', 'link', 'meta',
       'modified', 'modified_gmt', 'ping_status', 'slug', 'status', 'sticky',
       'tags', 'template', 'title', 'type'],
      dtype='object')

,_links,amp_validity,author,categories,comment_status,content,date,date_gmt,excerpt,featured_media,...,modified,modified_gmt,ping_status,slug,status,sticky,tags,template,title,type
59,{'self': [{'href': 'https://www.zambianobserve...,None,1,"[2, 12, 10, 14]",open,{'rendered': '<p>Gears Initiatives Zambia has ...,2019-05-27T13:45:57,2019-05-27T11:45:57,{'rendered': '<p>Gears Initiatives Zambia has ...,2305,...,2019-05-27T13:46:17,2019-05-27T11:46:17,open,ecz-called-upon-to-quickly-announce-dates-for-...,publish,False,[],,{'rendered': 'ECZ called upon to quickly annou...,post
52,{'self': [{'href': 'https://www.zambianobserve...,None,1,"[2, 12, 10, 14]",open,{'rendered': '<p>&nbsp;</p> <p>UPND Youths cou...,2019-05-27T13:51:05,2019-05-27T11:51:05,{'rendered': '<p>&nbsp; UPND Youths country wi...,200505,...,2019-05-27T13:51:05,2019-05-27T11:51:05,open,pf-tell-gbm-they-dont-want-him-with-his-vulgar...,publish,False,[],,{'rendered': 'PF Tell GBM They Don’t Want Him ...,post
22,{'self': [{'href': 'https://www.zambianobserve...,None,1,"[12, 10, 14]",open,{'rendered': '<p>National Democratic Congress ...,2019-05-28T09:03:04,2019-05-28T07:03:04,{'rendered': '<p>National Democratic Congress ...,121832,...,2019-05-28T09:03:04,2019-05-28T07:03:04,open,lungu-not-most-insulted-president-but-most-cor...,publish,False,[],,{'rendered': 'Lungu not most insulted Presiden...,post


#####                       Function to clean the contents, excerpts, guid and titles of the posts 

In [5]:
def clean_posts_data(column_title):
    content_list = []

    for i in data[column_title]:
        data_soup = soup(i['rendered'], 'html.parser')

        if bool(data_soup.find()):    # checking if the string has html tags
            results = data_soup.find_all('p')
            
            temp_string = ''
            for k in results:
                temp_string += (str(k.text))
            content_list.append(temp_string)
            
        else:
            results = data_soup.text

            temp_string = ''
            for k in results:
                temp_string += (str(k))
            content_list.append(temp_string)
            
    
    return content_list

# data.content = content_list

###### Cleaning contents of the posts

In [6]:
data.content = clean_posts_data('content')

##### Cleaning tiltes of the posts

In [7]:
data.title = clean_posts_data('title')

###### Cleaning 'excerpt' of the posts

In [8]:
data.excerpt = clean_posts_data('excerpt')

###### The cleaned dataFrame

In [11]:
data = data[['categories','content','excerpt','title']] #drop other columns
data.sample(5)

,categories,content,excerpt,title
18,"[2, 12, 10, 14]","Former UPND presidential advisor, Richard Kach...","Former UPND presidential advisor, Richard Kach...",Too Much Tribalism Made Me Ditch Upnd Charges ...
55,"[12, 10, 14]",South Africa President Cyril Ramaphosa is set...,South Africa President Cyril Ramaphosa is se...,Ramaphosa To Reduce Number Of Ministers While ...
97,"[53, 2, 12, 10, 14]",University of Lusaka Student Chokani Sakala w...,University of Lusaka Student Chokani Sakala ...,Sata’s Daughter Told Me She Was Allergic To Co...
74,"[12, 10, 14]",THE PF has done itself a great deal of harm by...,THE PF has done itself a great deal of harm by...,PF IS FACING A MOUNTAIN…no amount of repressi...
49,"[12, 10, 14]",The recently announced increase in the Monetar...,The recently announced increase in the Monetar...,Zambians to face hard economic times if fiscal...


##### Exporting to Excel file and it ('Excel file') will be save in the folder where the jupyter file is

In [10]:
# writer = pd.ExcelWriter('zambianobserver_posts.xlsx')
# data.to_excel(writer,sheet_name = 'Posts', engine='xlsxwriter')
# writer.save()